In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Définition du chemin vers les fichiers csv
path = "./transactions-ser"

# Récupération de tous les fichiers csv dans le répertoire spécifié
all_files = glob.glob(os.path.join(path, "*.csv"))
# Création d'une liste vide pour stocker tous les dataframes
all_df = []
# Boucle sur chaque fichier csv pour en créer un dataframe et l'ajouter à la liste
for f in all_files:
    df = pd.read_csv(f, sep=';')
    df['file'] = f.split('/')[-1]
    all_df.append(df)
    
# Concaténation de tous les dataframes en un seul
data = pd.concat(all_df, ignore_index=True)
merged_df = data.copy()
display(merged_df)

# Récupèrer que les lignes qui contient appartement et maison et les stockers dans des dataframe séparés
df_app = merged_df[merged_df.type_local == 'Appartement']
df_mai = merged_df[merged_df.type_local == 'Maison']

# Remplacement des valeurs 'None' par NaN dans les deux dataframes
df_app=df_app.replace(to_replace='None', value=np.nan)
df_mai=df_mai.replace(to_replace='None', value=np.nan)

df_mai


df_mai = df_mai.dropna(axis=1, how='all')

missing_rate = df_mai.isna().sum()/df_mai.shape[0]
missing_rate

df_mai = df_mai.drop(columns=['adresse_suffixe','lot1_numero', 'lot1_surface_carrez','nature_culture','code_type_local','nombre_lots','code_nature_culture_speciale','adresse_code_voie','code_postal','nature_culture_speciale','id_parcelle','type_local','nom_commune','adresse_nom_voie','id_mutation','file'
])

df_mai.corr()

df_mai = df_mai.drop(columns=['code_commune','code_departement'], axis=1)

sns.heatmap(df_mai.corr(), annot=True)
df_mai.numero_disposition.unique()

df_mai['date_mutation'] = pd.to_datetime(df_mai['date_mutation'])
df_mai['annee_mutation'] = df_mai['date_mutation'].dt.year

df_mai = df_mai.drop(['date_mutation'], axis=1)

df_mai.dtypes

code = {'Vente': 0,
        'Adjudication': 1,
        'Vente en l\'état futur d\'achèvement': 2,
        'S':0,
        'AG':1,
        'J':2,
        'L':3,
        'AB':4,
        '1':1,
        '2':2,
        '000AB': 0, '000AC': 1, '000AD': 2, '000AE': 3, '000AH': 4, '000AI': 5, '000AK': 6, '000AO': 7, '000AP': 8,
        '000AR': 9, '000AS': 10, '000AT': 11, '000AV': 12, '000AW': 13, '000AX': 14, '000AY': 15, '000AZ': 16, '000BC': 17,
        '000BD': 18, '000BE': 19, '000BH': 20, '000BK': 21, '000BL': 22, '000BM': 23, '000BO': 24, '000BP': 25, '000BR': 26,
        '000BS': 27, '000BT': 28, '000BV': 29, '000BW': 30
       }

for col in df_mai.select_dtypes('object'):
    df_mai[col] = df_mai[col].map(code)

display(df_mai)

plt.figure(figsize=(15, 10))
sns.heatmap(df_mai.corr(), annot=True)

df_mai = df_mai.drop(df_mai[df_mai['valeur_fonciere'] == 3060533.2].index)


df_mai = df_mai.drop(df_mai[df_mai['valeur_fonciere'] == 1.0].index)
sorted(df_mai.surface_terrain.values)



df_mai = df_mai.drop(df_mai[df_mai['nombre_pieces_principales'] == 22.0].index)
df_mai = df_mai.drop(columns=['numero_disposition','adresse_numero','section_prefixe','nature_mutation','annee_mutation'])


plt.figure(figsize=(15, 10))
sns.heatmap(df_mai.corr(), annot=True)

df_mai = df_mai.dropna(subset=['surface_terrain'])

df_mai = df_mai.drop(['code_nature_culture'], axis=1)

print(nan_counts)

df_mai

plt.scatter(df_mai.valeur_fonciere, df_mai.surface_reelle_bati)
sorted(df_mai.valeur_fonciere, reverse=True)

sns.heatmap(df_mai.corr(), annot=True)
df_mai.hist()

df_mai = df_mai.dropna()
df_mai

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline  import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

X = df_mai.drop(columns=['valeur_fonciere'], axis=1)
print(X)
y = df_mai['valeur_fonciere']

#poly_features = PolynomialFeatures(degree=degree, include_bias = False)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Create a polynomial transformer
poly = PolynomialFeatures(degree=2)

# Transform the training and test data
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
#poly_features = PolynomialFeatures(degree=degree, include_bias = False)
reg = LinearRegression()
#X_poly = poly_features.fit_transform(X_train)
#X_poly_test = poly_features.fit_transform(X_test)


reg.fit(X_train_poly,y_train)




from sklearn.metrics import mean_squared_error
predictions = reg.predict(X_test_poly)
mse = mean_squared_error(y_test, predictions)
print(mse)


reg.score(X_test_poly, y_test)

plt.scatter(y_test, predictions)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('Polynomial Regression Results')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)

results = pd.DataFrame({'True Values': y_test, 'Predicted Values': predictions})

# Print the first few rows of the results
print(results)

scores = cross_val_score(reg, X, y, cv=5)

print(np.mean(scores))

